In [1]:
import pandas as pd
from pytz import country_timezones
import time
import numpy as np
import timezonefinder
from datetime import datetime as dt
import pytz
import calendar
import seaborn as sns
%matplotlib inline
from matplotlib import rcParams

In [2]:
start_ini = time.time() #check time

In [3]:
import warnings
warnings.filterwarnings('ignore') #remove warning

In [4]:
#import dataset
df = pd.read_csv('I:/Massi/Documents/GitHub/NeoMeetup/Csv/Exported_queries/Distribuzione_temporale_eventi.csv')
df.head()

,Name,Time,Country,State,Lon,Lat
0,\tAnnual New Year's Resolution Hiking Pedernal...,05/01/2019 13:00:00,us,TX,-98.67,29.46
1,\tDance for Kids 7-12,03/01/2019 23:00:00,us,AZ,-110.89,32.22
2,\tEasy Four Mile Creek Greenway MATTHEWS,29/12/2018 18:30:00,us,NC,-80.78,35.29
3,\tGIRLS NIGHT OUT !!! MEN IN MOTION MALE REVIE...,17/02/2019 00:00:00,us,CT,-72.67,41.77
4,\tSpeak EZ Toastemasters,28/12/2018 00:00:00,us,TX,-96.75,32.96


In [5]:
df['State'] = df['State'].str.lower() #lower charcter 'NONE'-->'none'
df[df['State'] == 'none'].head()

,Name,Time,Country,State,Lon,Lat
10,NaN,27/01/2019 10:00:00,gb,none,-0.07,51.49
14,\n\u2018Call Me by Your Name\u2019 \u2013 Prin...,27/12/2018 15:15:00,gb,none,-0.1,51.52
15,\n\u2018Call Me by Your Name\u2019 \u2013 Prin...,27/12/2018 15:15:00,gb,none,-0.05,51.5
16,\n\u2018Shoplifters\u2019 \u2013 Prince Charle...,05/01/2019 14:50:00,gb,none,-0.4,51.67
17,\n\u2018Shoplifters\u2019 \u2013 Prince Charle...,05/01/2019 14:50:00,gb,none,-0.1,51.52


In [6]:
df.loc[df['State'] == 'none', 'State'] = df['Country'] #fill the state's value with country's value if there was 'none'

In [7]:
df.isnull().sum() #check sum of nan fro columns

Name        41
Time       102
Country    102
State      102
Lon        102
Lat        102
dtype: int64

In [8]:
df = df.dropna() #rm nan value for all df (rows)

In [9]:
df.isnull().any() #check

Name       False
Time       False
Country    False
State      False
Lon        False
Lat        False
dtype: bool

In [10]:
#df.loc[df['State'] == 'none', 'State'] = df.fillna(value=pd.np.nan, inplace=True) 
#it's only if would take nan into country that do not have a state

In [11]:
df.isnull().sum() #check

Name       0
Time       0
Country    0
State      0
Lon        0
Lat        0
dtype: int64

In [12]:
df.isnull().any() #another check :/

Name       False
Time       False
Country    False
State      False
Lon        False
Lat        False
dtype: bool

In [13]:
#same method for cheò nan value into coordinates
df['Lon'] = df['Lon'].str.lower()
df['Lat'] = df['Lat'].str.lower()
df.loc[df['Lon'] == 'none', 'Lon'] = df.fillna(value=pd.np.nan, inplace=True)
df.loc[df['Lat'] == 'none', 'Lat'] = df.fillna(value=pd.np.nan, inplace=True)
df.isnull().sum()

Name        0
Time        0
Country     0
State       0
Lon        48
Lat         9
dtype: int64

In [14]:
df = df.dropna() #remove this nan
df.isnull().sum()

Name       0
Time       0
Country    0
State      0
Lon        0
Lat        0
dtype: int64

In [15]:
df.tail() #df clean

,Name,Time,Country,State,Lon,Lat
211357,~~ 70's & 80's Saturday Night Dance Party ~~,13/01/2019 01:30:00,us,ny,-73.69,40.98
211358,~~ 70's & 80's Saturday Night Dance Party ~~,13/01/2019 01:30:00,us,ct,-73.45,41.39
211359,~~ 70's & 80's Saturday Night Dance Party ~~,13/01/2019 01:30:00,us,ny,-73.86,41.08
211360,~~ 70's & 80's Saturday Night Dance Party ~~,13/01/2019 01:30:00,us,ct,-73.63,41.04
211361,~~ HAPPY NEW YEAR 2019 Pinochle PLUS......Game...,13/01/2019 00:00:00,us,pa,-75.14,40


In [16]:
df['Lon'] = df['Lon'].astype(float) #convert coordinates into float for problem into cell below (timezonefinder)
df['Lat'] = df['Lat'].astype(float)
df.dtypes

Name        object
Time        object
Country     object
State       object
Lon        float64
Lat        float64
dtype: object

In [17]:
df = df[[c for c in df if c not in ['Lat','Lon']] + ['Lat','Lon']] #change order of column for best position of Lat and lng

In [18]:
df = df.reset_index(drop=True) #reset index for resolve eroor of out-bounded

In [19]:
tf = timezonefinder.TimezoneFinder() #defining time zones on lat, long data
df['timezone_str'] = "nan"
count = 0
start = time.time()
for line in df.itertuples():
    count+=1
    try:
        df.timezone_str.at[line.Index] = tf.timezone_at(lat = df.iloc[line.Index]['Lat'],
                                                          lng = df.iloc[line.Index]['Lon'])
    except Exception as e:
        print(e)
        break
    #if count == 10:
        #break
end = time.time()
print("task completed in "+str((end-start)/60)+" minutes")

task completed in 1.2866118709246317 minutes


In [20]:
df[50:70] #check

,Name,Time,Country,State,Lat,Lon,timezone_str
50,Lift your Spirits with Laughter! Join us fo...,05/01/2019 19:00:00,us,ny,43.17,-77.60,America/New_York
51,Look forward to seeing you next week!! No R...,04/01/2019 00:30:00,us,la,30.45,-91.06,America/Chicago
52,Meditation to Remove Addictions with Rene De...,29/01/2019 02:00:00,us,nv,36.19,-115.39,America/Los_Angeles
53,New Years Eve Invitation from the SHAHENSHAH...,01/01/2019 01:00:00,us,va,38.82,-77.29,America/New_York
54,New Years Eve Invitation from the SHAHENSHAH...,01/01/2019 01:00:00,us,md,39.29,-76.62,America/New_York
55,New Years Eve Invitation from the SHAHENSHAH...,01/01/2019 01:00:00,us,dc,38.91,-77.02,America/New_York
56,Nunsense @ Southside Theatre Guild,27/01/2019 19:30:00,us,ga,33.83,-84.39,America/New_York
57,Oratory Otters Club Meeting,01/01/2019 02:00:00,us,ca,37.39,-121.90,America/Los_Angeles
58,Peppercorns,29/01/2019 00:00:00,us,ny,40.69,-73.56,America/New_York
59,Post January Saturday Afternoon Pub Crawl,02/02/2019 12:30:00,gb,gb,52.48,-1.91,Europe/London


In [21]:
df.dtypes

Name             object
Time             object
Country          object
State            object
Lat             float64
Lon             float64
timezone_str     object
dtype: object

In [22]:
df['Time'] = pd.to_datetime(df['Time']) #convert time (object) into datettime type (needed fro convert after into timestamp)

In [23]:
df['timestamp'] = df.fillna(value=pd.np.nan, inplace=True) #fill with nan value
start = time.time()
count = 0
for line in df.itertuples():
    count+=1
    try:
        df['timestamp'].at[line.Index] = dt.timestamp(df.iloc[line.Index]['Time'])
    except Exception as e:
        print(e)
        break
    #if count == 100:
        #break
end = time.time()
print('task completed in '+str((end-start)/60)+' minutes')

task completed in 0.651246964931488 minutes


In [24]:
df.head(10) #check

,Name,Time,Country,State,Lat,Lon,timezone_str,timestamp
0,\tAnnual New Year's Resolution Hiking Pedernal...,2019-05-01 13:00:00,us,tx,29.46,-98.67,America/Chicago,1.55671e+09
1,\tDance for Kids 7-12,2019-03-01 23:00:00,us,az,32.22,-110.89,America/Phoenix,1.55148e+09
2,\tEasy Four Mile Creek Greenway MATTHEWS,2018-12-29 18:30:00,us,nc,35.29,-80.78,America/New_York,1.5461e+09
3,\tGIRLS NIGHT OUT !!! MEN IN MOTION MALE REVIE...,2019-02-17 00:00:00,us,ct,41.77,-72.67,America/New_York,1.55036e+09
4,\tSpeak EZ Toastemasters,2018-12-28 00:00:00,us,tx,32.96,-96.75,America/Chicago,1.54595e+09
5,\tSpeak EZ Toastemasters,2019-11-01 00:00:00,us,tx,32.96,-96.75,America/Chicago,1.57256e+09
6,\tSpecial Small Group Meetings - Off-Site Loca...,2019-01-25 12:45:00,us,tn,36.00,-86.79,America/Chicago,1.54842e+09
7,\tTech Center Park,2018-12-26 18:00:00,us,or,45.44,-122.77,America/Los_Angeles,1.54584e+09
8,"\tValuing People- Hal Hadden, facilitating",2019-01-18 12:45:00,us,tn,36.00,-86.79,America/Chicago,1.54781e+09
9,Shamanic Breathwork with Akasha NohAta,2018-12-29 00:00:00,us,pa,40.21,-75.08,America/New_York,1.54604e+09


In [25]:
def rounder(t): #this function round the time to nearest hour
    if t.minute >= 30:
        if t.hour == 23:
            return t.replace(second=0, microsecond=0, minute=0, hour=0)
        else:
            return t.replace(second=0, microsecond=0, minute=0, hour= t.hour+1)
    else:
        return t.replace(second=0, microsecond=0, minute=0)

In [26]:
df['datetime_timezoned'] = df.fillna(value=pd.np.nan, inplace=True) #fill with nan value

In [27]:
df['datetime_timezoned'] = df['datetime_timezoned'].astype(object) #convert for resolve error in the cell above (should not be a number)
df.dtypes

Name                          object
Time                  datetime64[ns]
Country                       object
State                         object
Lat                          float64
Lon                          float64
timezone_str                  object
timestamp                    float64
datetime_timezoned            object
dtype: object

In [28]:
start = time.time()
count = 0
for line in df.itertuples():
    count+=1
    try:
        df.datetime_timezoned.at[line.Index] = rounder((dt.fromtimestamp
                                                   (timestamp = (df.iloc[line.Index]['timestamp']),
                                                    tz = pytz.timezone(df.iloc[line.Index]['timezone_str'])
                                                   ))).strftime("%d/%m/%Y %H:%M:%S")
    except Exception as e:
        print(e)
        break
    #if count == 100:
        #break
end = time.time()
print('task completed in '+str((end-start)/60)+' minutes')

task completed in 1.3555020252863565 minutes


In [29]:
df = df[[c for c in df if c not in ['timestamp','Time','datetime_timezoned']] + ['timestamp','Time','datetime_timezoned']]
df.head(10) #re-order column

,Name,Country,State,Lat,Lon,timezone_str,timestamp,Time,datetime_timezoned
0,\tAnnual New Year's Resolution Hiking Pedernal...,us,tx,29.46,-98.67,America/Chicago,1.556708e+09,2019-05-01 13:00:00,01/05/2019 06:00:00
1,\tDance for Kids 7-12,us,az,32.22,-110.89,America/Phoenix,1.551478e+09,2019-03-01 23:00:00,01/03/2019 15:00:00
2,\tEasy Four Mile Creek Greenway MATTHEWS,us,nc,35.29,-80.78,America/New_York,1.546105e+09,2018-12-29 18:30:00,29/12/2018 13:00:00
3,\tGIRLS NIGHT OUT !!! MEN IN MOTION MALE REVIE...,us,ct,41.77,-72.67,America/New_York,1.550358e+09,2019-02-17 00:00:00,16/02/2019 18:00:00
4,\tSpeak EZ Toastemasters,us,tx,32.96,-96.75,America/Chicago,1.545952e+09,2018-12-28 00:00:00,27/12/2018 17:00:00
5,\tSpeak EZ Toastemasters,us,tx,32.96,-96.75,America/Chicago,1.572563e+09,2019-11-01 00:00:00,31/10/2019 18:00:00
6,\tSpecial Small Group Meetings - Off-Site Loca...,us,tn,36.00,-86.79,America/Chicago,1.548417e+09,2019-01-25 12:45:00,25/01/2019 06:00:00
7,\tTech Center Park,us,or,45.44,-122.77,America/Los_Angeles,1.545844e+09,2018-12-26 18:00:00,26/12/2018 09:00:00
8,"\tValuing People- Hal Hadden, facilitating",us,tn,36.00,-86.79,America/Chicago,1.547812e+09,2019-01-18 12:45:00,18/01/2019 06:00:00
9,Shamanic Breathwork with Akasha NohAta,us,pa,40.21,-75.08,America/New_York,1.546038e+09,2018-12-29 00:00:00,28/12/2018 18:00:00


In [30]:
df.to_csv('I:/Massi/Documents/GitHub/NeoMeetup/Csv/Exported_queries/Distribuzione_temporale_eventi_fix.csv') #csv

In [31]:
end_fin = time.time()
print("script completed in "+str((end_fin-start_ini)/60)+" minutes")

script completed in 3.7282972812652586 minutes
